Sharefood scraping

In [0]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
urllist = []
url = "https://www.sharefood.sg/"
response = get(url, headers = headers)
html_soup = BeautifulSoup(response.text, 'html.parser')
data = html_soup.find('div', class_ = 'penci-mega-child-categories')
urls = data.find_all('a')
for u in urls[3:]:
  urllist.append(u.get('href'))

In [0]:
!pip install xlsxwriter

     |████████████████████████████████| 143kB 9.8MB/s 


In [0]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import xlsxwriter
import re
from openpyxl import load_workbook
from datetime import datetime


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
url_list,name_list, cuisine_list,servings,date_list, prep_list,cook_list,inst_list,main_ing = ([] for i in range(9))
for url in urllist:
    response = get(url, headers = headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    data = html_soup.find_all('h2', class_ = 'entry-title grid-title')
    for d in data:
        url = d.find('a').get('href')
        match = re.search(r'\d{4}/\d{2}/\d{2}', url)
        date = datetime.strptime(match.group(), '%Y/%m/%d').date()
        if(date.year >= 2019):
          url_list.append(url)
          date_list.append(date)
          response = get(url, headers = headers)
          html_soup = BeautifulSoup(response.text, 'html.parser')
          recipe_name = html_soup.find('span',class_ = 'wpurp-recipe-title')
          if(recipe_name is not None):
              name_list.append(recipe_name.get_text())
          else:
              name_list.append("NA")
          cuisine = html_soup.find('span',class_= 'wpurp-recipe-tag-terms')
          if(cuisine is not None):
              cuisine_list.append(cuisine.get_text())
          else:
              cuisine_list.append("NA")
          p = html_soup.find('span',class_ = 'wpurp-recipe-servings')
          if(p is not None):
              type = html_soup.find('span',class_ = 'wpurp-recipe-servings-type')
              servings.append(p.get_text() + type.get_text())
          else:
              servings.append("NA")
          prep_time = html_soup.find('span',class_= 'wpurp-recipe-prep-time')
          if(prep_time is not None):
              time_text = html_soup.find('span',class_ = 'wpurp-recipe-prep-time-text').get_text()
              prep_list.append(prep_time.get_text() + time_text)
          else:
              prep_list.append("NA")
          cook_time = html_soup.find('span', class_ = 'wpurp-recipe-cook-time')
          if(cook_time is not None):
              cooktime_text = html_soup.find('span', class_ = 'wpurp-recipe-cook-time-text').get_text()
              cook_list.append(cook_time.get_text() + cooktime_text)
          else:
              cook_list.append("NA")
          ingredients_group = html_soup.find_all('div', class_ = 'wpurp-recipe-ingredient-group-container')
          ing_list = []
          for ing_group in ingredients_group:
            group = ing_group.find('span', class_ = 'wpurp-recipe-ingredient-group')
            if(group is not None):
              group = group.get_text()
            else:
              group = ""
            ingredients = ing_group.find('ul', class_ = 'wpurp-recipe-ingredient-container')
            if(ingredients is not None):
                    ing = ingredients.find_all('li')
                    for each in ing:
                      ing_name = each.find('span',class_ = 'wpurp-recipe-ingredient-name').get_text()
                      ing_name.replace('-','')
                      ing_quantity = each.find('span',class_ = 'wpurp-recipe-ingredient-quantity')
                      if(ing_quantity is not None):
                        ing_quantity = ing_quantity.get_text()
                      else:
                        ing_quantity = ""
                      ing_unit = each.find('span',class_ = 'wpurp-recipe-ingredient-unit')
                      if(ing_unit is not None):
                        ing_unit = ing_unit.get_text()
                      else:
                        ing_unit = ""
                      ing_notes = each.find('span', class_ = 'wpurp-recipe-ingredient-notes')
                      if(ing_notes is not None):
                        ing_notes = ing_notes.get_text()
                        ing_notes.replace('-','')
                        ing_list.append(group+"-"+ ing_quantity+"-"+ing_unit+"-"+ing_name +"-"+ ing_notes)
                      else:
                        ing_list.append(group+"-"+ ing_quantity+"-"+ing_unit+"-"+ing_name +"-")
            else:
              ing_list.append("NA")
          main_ing.append(ing_list)
          instructions = html_soup.find('ol',class_ = 'wpurp-recipe-instruction-container')
          if(instructions is not None):
              inst_list.append(instructions.get_text().strip())
          else:
              inst_list.append("NA")
df = pd.DataFrame({'url':url_list,
                      'date':date_list,
                      'recipename':name_list,
                      'cuisine':cuisine_list,
                      'servings':servings,
                      'preptime':prep_list,
                      'cookingtime':cook_list,
                      'ingredients':main_ing,
                      'instructions':inst_list})
# writer = pd.ExcelWriter('output.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
# df.to_excel(writer,'Sheet1')
# writer.save()

In [0]:
import pandas as pd
for i in df.index:
  df['ingredients'][i] = df['ingredients'][i].replace(',','\n')
  df['ingredients'][i] = df['ingredients'][i].replace("'",'')
  df['ingredients'][i] = df['ingredients'][i].replace('[','')
  df['ingredients'][i] = df['ingredients'][i].replace(']','')
df.to_csv ('output2.csv', index = None, header=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set 

Check allergies

In [0]:
allergies = {
    1: ["beef","beef rib","worcestershire sauce"],
    2: ["Milk","Butter","Cheese","Cream","Yogurt","Custard","Curd","Paneer","Ghee","So (jap)","Whey","Yakult","Chocolate","Margarine","Horseradish Sauce","Mayo"],
    3: ["Egg","Egg Yolk","Egg White","Pasta","Mayonnaise","Custard","Mayo"],
    4: ["Wheat flour","All purpose flour","all-purpose-flour","Lo Mein","Stir Fry Sauce","Udon","Ramen","imitation crab","Wanton noodles","Miso paste","Miso","Plum sauce","Yakisoba","Wasabi","Oats","Broth","Soy sauce","Teriyaki sauce","Maltose","Fu","Hoisin sauce","Oyster sauce","Shaoxing wine","Ketchup","Mayonnaise","Seitan","Roasted nuts","Knorr","Maggi Seasoning sauce","Chicken stock powder","Malt vinegar","Vodka","Ebelskiver mix" ,"Korean red pepper paste (gochuchang)"],
    5: ["Soy bean","soya bean", "Almond","Brazil nut","Peanut","Cashew","Chestnut","Hazelnut","Macadamia nut","Pecan","Pistachio","Shea nut","Walnut","Pesto","Coconut","Gingko nut","Hickory nut","Pine nut","Nangai nut","Knorr seasoning","beer nut","Belgian ale","brazil nut", "chop nut","cocktail peanut", "ginkgo nut","grand marnier","roundnut","hazelnut","macadamia nut","butter","nutmeg","pecan","pine nut","pistachio","Planters Cocktail Peanut","red skin peanut","tamari almond","trail mix","unsalted sunflower seed","walnut"],
    6: ["Gelatin","Pork" ,"Lard","Ham","Bacon","Cooking fat","pork rib"],
    7: ["abalone","banana prawn","bay scallop","bay shrimp","blood cockle","calamari ring","calamari steak", "clam","cockle","crab","crawfish","cuttlefish","dashima","shrimp","bonito", "cuttlefish","kelp","kombu","lobster" ,"mussels", "oyster", "rock lobster", "scallop","scallopini","scampi","shellfish","albacore tuna","Fish sauce","Fish paste", "Clams","Squid","Mussels","Prawns","Sea urchin","Limpet","Octopus","Snails","Surimi","Langoustine","Sambal goreng paste"],
    8: ["Hoisin sauce","Stir Fry Sauce","Oyster sauce","Douchi","Tamari","Teriyaki sauce","Miso","Lao Gan Ma","Bean curd","chai pow yu", "tofu","doufu","egg substitute","Ener-G Egg Replacer","imitation crab","meat substitute","quorn","Quorn Chik''n Tender","Quorn fillet", "soy bean", "soy", "yogurt","soya bean","soybean paste","soynut", "tau kwa","tempeh","tvp","vegetarian beef","veggie sausage","yuba","tau pok"],
    9: ["Gula melaka","Curry paste","Sugar","Sambal goreng paste","Ketchup","BBQ Sauce","Sriracha","Sweet","Hoisin" ,"Teriyaki","Balsamic Vinegar"]
}

In [0]:
#converting ingredients to lowercase
def recipe_lowercase():
  import pandas as pd
  fields = ['ingredients']
  df = pd.read_csv('output2.csv',usecols = fields)
  return df

In [0]:
#converting allergies to lowercase
newdict = {}
for key, value in allergies.items():
  newdict[key] = [v.lower() for v in value]

In [0]:
#final check once
import pandas as pd
from collections import defaultdict

dict1 = defaultdict(list)
df = recipe_lowercase()
for i in range(len(df)):
  flag = 0
  for key, value in newdict.items():
    if (pd.notna(df['ingredients'][i])):
      if any (v in df['ingredients'][i] for v in value):
        flag = 1
        dict1[i].append(key)
      else:
        dict1[i].append("NA")
    else:
      dict1[i].append("NA")
allergies_list = list(dict1.values())
print(allergies_list)

Check kitchen equipment

In [0]:
equip = {
    1: "microwave",
    2: "oven",
    3: "blender",
    4: "rice cooker",
    5: "slow cooker",
    6: "steamer",
    7: "food processor",
}

In [0]:
#converting recipes to lowercase, can be used in cooking method as well
def recipe_lowercase():
  import pandas as pd
  df1 = []
  df = pd.read_excel('yoripe_25sept.xlsx', sheet_name = 'Yoripe', usecols = 'I')
  for i in df.index:
    df1.append(df['instructions'][i])
  return df1

In [0]:
import pandas as pd
from collections import defaultdict

newdict = defaultdict(list)
df = recipe_lowercase()
for i in range(len(df)):
  flag = 0
  for key, value in equip.items():
    if (pd.notna(df[i])):
      if value not in df[i]:
        newdict[i].append("NA")
      else:
        flag = 1
        newdict[i].append(key) 
    else:
      newdict[i].append("NA")
print (newdict)

check cooking method

In [0]:
method = {
    1 :["stir-fry", "stir fry"],
    2: ["pan-fry", "pan fry"],
    3: ["deep-fry","deep fry"],
    4: "boil",
    5: "roast",
    6: "bake",
    7:  "boil",
    8: "steam", 
    9: "grill",
    10: "braise",
    11:"poach",
    12: "sear",
    13: ["saute", "sauté"],
    14: "microwave",
    15: "stew",
    16: "blanch",
    17: "simmer",
}

In [0]:
#final check once
import pandas as pd
from collections import defaultdict

newdict = defaultdict(list)
df = recipe_lowercase()
print (df)
for i in range(len(df)):
  flag = 0
  for key, value in method.items():
    if (pd.notna(df[i])):
      if isinstance(value, list):
        if any (v in df[i] for v in value):
          flag = 1
          newdict[i].append(key)
      elif (value in df[i]):
          flag = 1
          newdict[i].append(key)
      else:
        newdict[i].append("NA")
    else:
      newdict[i].append("NA")
print (newdict)
  